**My Strategy**


---

I take the provided dictionary of 250,000 words and generate my dataset from it. To do so, for each word, I randomly remove one or more letters from it, and replace them with blanks. For example, from apple I can create a__le , app_e , _pp__ , and so on. I randomly create 4-8 such training words from each word in the dictionary. For each of these words I also store the corresponding target word.
The model I have used contains 2 Bi-directional LSTM layers with dropouts. The final output layer is a dense layer which predicts one of 26 letters. While training the model, the word with blanks is the input and the letter to be guessed is the expected output. If 2 or more letters are missing, then while training, I pass that particular example that many times, the expected output being one of the missing letters each time. For example, if we have app__, once we will pass app__ with expected output 'l' , and once we will pass app__ with expected output 'e'.

I have trained the model using just 1 epoch, due to space and time contraints on colab free tier. I have also not generated a very large dataset or made the model very complex for the same reason (I tried some combinations, but colab was crashing). The accuracy can perhaps be improved with more layers and more training.

Note: I have not sourced my solution from the internet in any way. It is 100% original.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

# Load training data
with open('/content/drive/MyDrive/words_250000_train.txt', 'r') as file:
    words = file.read().splitlines()

def generate_gaps(word):
    n = len(word)
    gap_versions = []
    num_versions = random.randint(4, 8)
    for _ in range(num_versions):  # Generate between 4 and 8 gaps
        gap_word = list(word)
        target_word = list('*' * n)

        unique_letters = list(set(word))
        num_gaps = random.randint(1, len(unique_letters))  # Random number of unique letters to replace
        letters_to_replace = random.sample(unique_letters, num_gaps)

        for letter in letters_to_replace:
            for i in range(n):
                if word[i] == letter:
                    gap_word[i] = '_'
                    target_word[i] = letter

        gap_versions.append((''.join(gap_word), ''.join(target_word)))
    return gap_versions


data = []
for word in words:
    data.extend(generate_gaps(word))

In [4]:
random.shuffle(data)

In [5]:
# Prepare the dataset
chars = set(''.join([pair[0] + pair[1] for pair in data]))
char2idx = {char: idx for idx, char in enumerate(sorted(chars))}
idx2char = {idx: char for char, idx in char2idx.items()}

max_length = max(len(pair[0]) for pair in data)

def encode_seq(seq):
    return [char2idx[char] for char in seq]

def decode_char(idx):
    return idx2char[idx]

# Generate training samples for each missing character
X = []
y = []

for word, target in data:
    for i, char in enumerate(word):
        if char == '_':
            x_seq = encode_seq(word)
            X.append(x_seq)
            y.append(char2idx[target[i]])

X = pad_sequences(X, maxlen=max_length, padding='post')
y = np.array(y)
y = to_categorical(y, num_classes=len(char2idx))


In [6]:
from tensorflow.keras.layers import Dropout

# Model
input = Input(shape=(max_length,))
embedding = Embedding(input_dim=len(char2idx), output_dim=128, input_length=max_length)(input)
bi_lstm1 = Bidirectional(LSTM(128, return_sequences=True))(embedding)
dropout1 = Dropout(0.2)(bi_lstm1)
bi_lstm2 = Bidirectional(LSTM(128))(dropout1)
dropout2 = Dropout(0.2)(bi_lstm2)
output = Dense(len(char2idx), activation='softmax')(dropout2)

model = Model(inputs=input, outputs=output)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 29)]              0         
                                                                 
 embedding (Embedding)       (None, 29, 128)           3584      
                                                                 
 bidirectional (Bidirection  (None, 29, 256)           263168    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 29, 256)           0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 256)               394240    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 256)               0     

In [7]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=1, batch_size=32, validation_split=0.2)

80641/80641 [==============================] - 1031s 13ms/step - loss: 2.6786 - accuracy: 0.1682 - val_loss: 2.6220 - val_accuracy: 0.1808


In [8]:
from tensorflow.keras.models import save_model

# Example: Assuming `model` is your trained Keras model
model.save('hangman_model.h5')

from google.colab import files
files.download('hangman_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import requests
import time
import collections
from urllib.parse import parse_qs

# Load the trained model
model = load_model('hangman_model.h5')

# Assuming you have char2idx and idx2char mappings already defined
# char2idx = {...}
# idx2char = {...}

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "/content/drive/MyDrive/words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.current_dictionary = []

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        # Function to guess the next letter in the word
        word = word.replace(' ' , '')

        # Encode the current word to numerical sequence
        encoded_word = [char2idx[char] if char in char2idx else char2idx['_'] for char in word]
        encoded_word = np.array([encoded_word])
        encoded_word = pad_sequences(encoded_word, maxlen=max_length, padding='post')

        # Predict the next letter
        predictions = model.predict(encoded_word)
        predicted_char_idx = np.argmax(predictions, axis=-1)[0]
        predicted_char = idx2char[predicted_char_idx]

        # Ensure the guessed character is not already guessed
        while predicted_char in self.guessed_letters:
            predictions[0][predicted_char_idx] = 0  # Zero out the current prediction
            predicted_char_idx = np.argmax(predictions, axis=-1)[0]
            predicted_char = idx2char[predicted_char_idx]

        return predicted_char

    ##########################################################
    # You'll likely not need to modify any of the code below #
    # return guessed letter
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import requests
import time
import collections
from urllib.parse import parse_qs

# Load the trained model
model = load_model('hangman_model.h5')

# Assuming you have char2idx and idx2char mappings already defined
# char2idx = {...}
# idx2char = {...}

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "/content/drive/MyDrive/words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.current_dictionary = []

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        # Function to guess the next letter in the word
        word = word.replace(' ' , '')

        # Encode the current word to numerical sequence
        encoded_word = [char2idx[char] if char in char2idx else char2idx['_'] for char in word]
        encoded_word = np.array([encoded_word])
        encoded_word = pad_sequences(encoded_word, maxlen=max_length, padding='post')

        # Predict the next letter
        predictions = model.predict(encoded_word)
        predicted_char_idx = np.argmax(predictions, axis=-1)[0]
        predicted_char = idx2char[predicted_char_idx]

        # Ensure the guessed character is not already guessed
        while predicted_char in self.guessed_letters:
            predictions[0][predicted_char_idx] = 0  # Zero out the current prediction
            predicted_char_idx = np.argmax(predictions, axis=-1)[0]
            predicted_char = idx2char[predicted_char_idx]

        return predicted_char

    ##########################################################
    # You'll likely not need to modify any of the code below #
    # return guessed letter
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)


In [10]:
api = HangmanAPI(access_token="ENTER_YOUR_ACCESS_TOKEN", timeout=2000)


In [15]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sg.trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Successfully start a new game! Game ID: 55b8d2deaa4b. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ .
1/1 [==============================] - 0s 20ms/step
Guessing letter: e


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sg.trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Sever response: {'game_id': '55b8d2deaa4b', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ _ _ _ _ '}
1/1 [==============================] - 0s 19ms/step
Guessing letter: i


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sg.trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

Sever response: {'game_id': '55b8d2deaa4b', 'status': 'ongoing', 'tries_remains': 1, 'word': 'g o n o c h o r i s _ u s '}
1/1 [==============================] - 0s 27ms/step
Guessing letter: m


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sg.trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Sever response: {'game_id': '55b8d2deaa4b', 'status': 'success', 'tries_remains': 1, 'word': 'g o n o c h o r i s m u s '}
Successfully finished game: 55b8d2deaa4b


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sg.trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


run 445 practice games out of an allotted 100,000. practice success rate so far = 0.133


In [16]:
num_games = 50
num_wins = 0

for _ in range(num_games):
    if api.start_game(practice=1, verbose=True):
        num_wins += 1

print(f"Played {num_games} games with {num_wins} wins.")

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sg.trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Successfully start a new game! Game ID: c2735b90f5ae. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ .
1/1 [==============================] - 0s 19ms/step
Guessing letter: e


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sg.trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Sever response: {'game_id': 'c2735b90f5ae', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ _ '}
1/1 [==============================] - 0s 20ms/step
Guessing letter: a


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sg.trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

Sever response: {'game_id': '5787ae0ee57a', 'status': 'ongoing', 'tries_remains': 6, 'word': 'a n t e s _ r i _ t '}
1/1 [==============================] - 0s 20ms/step
Guessing letter: c


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sg.trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Sever response: {'game_id': '5787ae0ee57a', 'status': 'ongoing', 'tries_remains': 6, 'word': 'a n t e s c r i _ t '}
1/1 [==============================] - 0s 21ms/step
Guessing letter: p


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sg.trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Sever response: {'game_id': '5787ae0ee57a', 'status': 'success', 'tries_remains': 6, 'word': 'a n t e s c r i p t '}
Successfully finished game: 5787ae0ee57a
Played 50 games with 16 wins.


In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)

    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sg.trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


1/1 [==============================] - 0s 21ms/step


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sg.trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


1/1 [==============================] - 0s 19ms/step


...

1/1 [==============================] - 0s 20ms/step


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sg.trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


1/1 [==============================] - 0s 20ms/step


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sg.trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


1/1 [==============================] - 0s 20ms/step
